# Content Design for RAG
This notebook is part of a collection of material related to content design principles for retrieval-augmented generation (RAG).

You can explore the complete collection here: [Content Design for RAG on GitHub](https://github.com/spackows/ICAAI-2024_RAG-CD/blob/main/README.md)

**Example scenario**

Imagine your company sells seeds and gardening supplies online.  On your website, you have articles with gardening information and advice.  You are building a RAG solution for your company website that can answer customer questions about your products, using your website articles as a knowledge base.

# Identify FAQs
There are advantages to returning curated answers to frequently asked questions (FAQs) instead of using a large language model (LLM) to generate the same answers over and over.

This sample notebook demonstrates a simple approach this problem: using string comparison Python libraries.

**Contents**
1. FAQs
2. String comparison methods
3. Test matching sample questions

## 1. FAQs
Imagine you have curated the following frequently asked questions:

In [7]:
g_curated_answers = [
    {
        "id" : "curated_answer_01",
        "question_txt" : "How tall do cucumbers grow?",
        "answer_txt" : "Cucumber plants can grow as high as 6 feet",
        "generated_date" : "2024-06-13",
        "grounding_article" : { "id" : "article_123", "url" : "http://whatever123", "title" : "All about cucumbers" }
    },
    {
        "id" : "curated_answer_02",
        "question_txt" : "Can I grow tomatoes in containers",
        "answer_txt" : "Most tomato plants do well in containers.",
        "generated_date" : "2024-05-01",
        "grounding_article" : { "id" : "article_456", "url" : "http://whatever456",  "title" : "Container gardening" }
    }
]

And imagine the following questions come in at run time:

In [8]:
g_run_time_questions = [
    { "question_txt": "Can tomatoes be grown in pots?",                 "expected_match" : "curated_answer_02" },
    { "question_txt": "How big are cucumber plants",                    "expected_match" : "curated_answer_01" },
    { "question_txt": "What size containers are needed for cucumbers?", "expected_match" : None },
    { "question_txt": "How big a pot can I use for tomatoes?",          "expected_match" : None }
]

## 2. String comparison methods
There are many ways to compare strings:
- 2.1 Fuzzy string matching 
- 2.2 Semantic similarity
- 2.3 Text distance
- 2.4 Normalizing text

### 2.1 Fuzzy string matching
Here are links to articles about this task:
- https://towardsdatascience.com/natural-language-processing-for-fuzzy-string-matching-with-python-6632b7824c49
- https://pypi.org/project/fuzzywuzzy/
- https://en.wikipedia.org/wiki/Levenshtein_distance
- https://www.datacamp.com/tutorial/fuzzy-string-python
- https://github.com/seatgeek/thefuzz

In [ ]:
!pip install thefuzz | tail -n 1

In [9]:
from thefuzz import fuzz
import statistics
from operator import itemgetter

def getFuzzScores( run_time_question, existing_questions_arr ):
    all_scores_arr = []
    for i in range( len( existing_questions_arr ) ):
        existing_question = existing_questions_arr[i]["question_txt"]
        answer_id = existing_questions_arr[i]["id"]
        s1 = fuzz.ratio( existing_question, run_time_question )
        s2 = fuzz.partial_ratio( existing_question, run_time_question )
        s3 = fuzz.token_sort_ratio( existing_question, run_time_question )
        s4 = fuzz.token_set_ratio( existing_question, run_time_question )
        s5 = fuzz.partial_token_sort_ratio( existing_question, run_time_question )
        scores = [ s1, s2, s3, s4, s5 ]
        ave = statistics.mean( scores )
        all_scores_arr.append( scores + [ ave, existing_question, answer_id ] )
    all_scores_arr = list( reversed( sorted( all_scores_arr, key=itemgetter(5) ) ) )
    result_json = []
    for row in all_scores_arr:
        result_json.append( { "curated_id" : row[7], 
                              "s1"         : row[0],
                              "s2"         : row[1],
                              "s3"         : row[2],
                              "s4"         : row[3],
                              "s5"         : row[4],
                              "ave_score"  : int( row[5] ),
                              "curated_question_txt" : row[6] } )
    return result_json

In [10]:
import pandas as pd
run_time_question = g_run_time_questions[0]["question_txt"]
results = getFuzzScores( run_time_question, g_curated_answers )
print( "Run-time question:\n\"" + run_time_question + "\"\n" )
print( "Match scores:" )
pd.DataFrame( results )

Run-time question:
"Can tomatoes be grown in pots?"

Match scores:


,curated_id,s1,s2,s3,s4,s5,ave_score,curated_question_txt
0,curated_answer_02,60,63,68,74,71,67,Can I grow tomatoes in containers
1,curated_answer_01,46,51,47,47,51,48,How tall do cucumbers grow?


### 2.2 Semantic similarity
Here are links to articles about this task:
- https://medium.com/georgian-impact-blog/an-introduction-to-semantic-matching-techniques-in-nlp-and-computer-vision-c22bf3cee8e9
- https://www.sbert.net/docs/quickstart.html
- https://www.sbert.net/docs/usage/semantic_textual_similarity.html
- https://www.sbert.net/examples/applications/paraphrase-mining/README.html
- https://www.sbert.net/examples/applications/clustering/README.html

In [ ]:
!pip install sentence-transformers | tail -n 1

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
import numpy as np

st_model = SentenceTransformer('all-MiniLM-L6-v2')

def getSentenceTransformerScores( run_time_question, currated_answers_arr ):
    existing_questions_arr = []
    for obj in currated_answers_arr:
        existing_questions_arr.append( obj["question_txt"] )
    run_time_question_arr = [ run_time_question ] * len( existing_questions_arr )
    run_time_question_embeddings  = st_model.encode( run_time_question_arr,  convert_to_tensor=True )
    existing_questions_embeddings = st_model.encode( existing_questions_arr, convert_to_tensor=True )
    cosine_scores = util.cos_sim( run_time_question_embeddings, existing_questions_embeddings )
    sentence_transformers_score_arr = [ round( float( x ), 2 ) for x in cosine_scores[0] ]
    ordered_index = list( reversed( np.argsort( sentence_transformers_score_arr ) ) )
    result_json = []
    for index in ordered_index:
        result_json.append( { "curated_id" : currated_answers_arr[index]["id"], 
                              "score"   : int( 100*sentence_transformers_score_arr[index] ),
                              "curated_question_txt" : existing_questions_arr[index] } )
    return result_json

In [14]:
run_time_question = g_run_time_questions[0]["question_txt"]
results = getSentenceTransformerScores( run_time_question, g_curated_answers )
print( "Run-time question:\n\"" + run_time_question + "\"\n" )
print( "Match scores:" )
pd.DataFrame( results )

Run-time question:
"Can tomatoes be grown in pots?"

Match scores:


,curated_id,score,curated_question_txt
0,curated_answer_02,79,Can I grow tomatoes in containers
1,curated_answer_01,37,How tall do cucumbers grow?


### 2.3 Text distance
Here are links to articles about this task:
- https://pypi.org/project/textdistance
- https://yassineelkhal.medium.com/the-complete-guide-to-string-similarity-algorithms-1290ad07c6b7

In [ ]:
!pip install textdistance | tail -n 1

In [16]:
import textdistance as td

def gettextdistanceScores( run_time_question, existing_questions_arr ):
    all_scores_arr = []
    for i in range( len( existing_questions_arr ) ):
        existing_question = existing_questions_arr[i]["question_txt"]
        answer_id = existing_questions_arr[i]["id"]
        s01 = td.hamming.normalized_similarity( existing_question, run_time_question )
        #s02 = td.mlipns.normalized_similarity( existing_question, run_time_question )
        s03 = td.levenshtein.normalized_similarity( existing_question, run_time_question )
        s04 = td.damerau_levenshtein.normalized_similarity( existing_question, run_time_question )
        s05 = td.jaro_winkler.normalized_similarity( existing_question, run_time_question )
        s06 = td.jaro.normalized_similarity( existing_question, run_time_question )
        s07 = td.strcmp95.normalized_similarity( existing_question, run_time_question )
        s08 = td.needleman_wunsch.normalized_similarity( existing_question, run_time_question )
        s09 = td.gotoh.normalized_similarity( existing_question, run_time_question )
        s10 = td.smith_waterman.normalized_similarity( existing_question, run_time_question )
        s11 = td.jaccard.normalized_similarity( existing_question, run_time_question )
        s12 = td.sorensen.normalized_similarity( existing_question, run_time_question )
        s13 = td.sorensen_dice.normalized_similarity( existing_question, run_time_question )
        #s14 = td.dice.normalized_similarity( existing_question, run_time_question )
        s15 = td.tversky.normalized_similarity( existing_question, run_time_question )
        s16 = td.overlap.normalized_similarity( existing_question, run_time_question )
        #s17 = td.tanimoto.normalized_similarity( existing_question, run_time_question )
        s18 = td.cosine.normalized_similarity( existing_question, run_time_question )
        #s19 = td.monge_elkan.normalized_similarity( existing_question, run_time_question )
        s20 = td.bag.normalized_similarity( existing_question, run_time_question )
        s21 = td.lcsseq.normalized_similarity( existing_question, run_time_question )
        s22 = td.lcsstr.normalized_similarity( existing_question, run_time_question )
        s23 = td.ratcliff_obershelp.normalized_similarity( existing_question, run_time_question )
        #s24 = td.arith_ncd.normalized_similarity( existing_question, run_time_question )
        #s25 = td.rle_ncd.normalized_similarity( existing_question, run_time_question )
        #s26 = td.bwtrle_ncd.normalized_similarity( existing_question, run_time_question )
        s27 = td.sqrt_ncd.normalized_similarity( existing_question, run_time_question )
        s28 = td.entropy_ncd.normalized_similarity( existing_question, run_time_question )
        s29 = td.bz2_ncd.normalized_similarity( existing_question, run_time_question )
        s30 = td.lzma_ncd.normalized_similarity( existing_question, run_time_question )
        s31 = td.zlib_ncd.normalized_similarity( existing_question, run_time_question )
        s32 = td.mra.normalized_similarity( existing_question, run_time_question )
        s33 = td.editex.normalized_similarity( existing_question, run_time_question )

        scores = [      s01,      s03, s04, s05, s06, s07, s08, s09,
                   s10, s11, s12, s13,      s15, s16,      s18,
                   s20, s21, s22, s23,                s27, s28, s29,
                   s30, s31, s32, s33
                 ]
        scores = [ round( float( x ), 2 ) for x in scores ]
        ave = round( statistics.mean( scores ), 2 )
        all_scores_arr.append( [ answer_id, ave, existing_question ] + scores  )
    all_scores_arr = list( reversed( sorted( all_scores_arr, key=itemgetter(1) ) ) )
    #print( all_scores_arr )
    result_json = []
    for row in all_scores_arr:
        result_json.append( { "curated_id" : row[0], 
                              "s01"        : int( 100*row[3] ),
                              "s03"        : int( 100*row[4] ),
                              "s04"        : int( 100*row[5] ),
                              "s05"        : int( 100*row[6] ),
                              "s06"        : int( 100*row[7] ),
                              "s07"        : int( 100*row[8] ),
                              "s08"        : int( 100*row[9] ),
                              "s09"        : int( 100*row[10] ),
                              "s10"        : int( 100*row[11] ),
                              "s11"        : int( 100*row[12] ),
                              "s12"        : int( 100*row[13] ),
                              "s13"        : int( 100*row[14] ),
                              "s15"        : int( 100*row[15] ),
                              "s16"        : int( 100*row[16] ),
                              "s18"        : int( 100*row[17] ),
                              "s20"        : int( 100*row[18] ),
                              "s21"        : int( 100*row[19] ),
                              "s22"        : int( 100*row[20] ),
                              "s23"        : int( 100*row[21] ),
                              "s27"        : int( 100*row[22] ),
                              "s28"        : int( 100*row[23] ),
                              "s29"        : int( 100*row[24] ),
                              "s30"        : int( 100*row[25] ),
                              "s31"        : int( 100*row[26] ),
                              "s32"        : int( 100*row[27] ),
                              "s33"        : int( 100*row[28] ),
                              "ave_score"   : int( 100*row[1] ),
                              "curatedquestion_txt" : row[2] } )
    return result_json

In [17]:
run_time_question = g_run_time_questions[0]["question_txt"]
results = gettextdistanceScores( run_time_question, g_curated_answers )
print( "Run-time question:\n\"" + run_time_question + "\"\n" )
print( "Match scores:" )
df = pd.DataFrame( results )
df.iloc[:, 0:16]

Run-time question:
"Can tomatoes be grown in pots?"

Match scores:


,curated_id,s01,s03,s04,s05,s06,s07,s08,s09,s10,s11,s12,s13,s15,s16,s18
0,curated_answer_02,12,42,42,85,76,86,61,69,37,75,86,86,75,90,86
1,curated_answer_01,10,20,20,61,61,64,53,56,19,43,60,60,43,63,60


In [18]:
df.iloc[:, 16:]

,s20,s21,s22,s23,s27,s28,s29,s30,s31,s32,s33,ave_score,curatedquestion_txt
0,82,57,30,60,51,98,62,69,51,50,45,64,Can I grow tomatoes in containers
1,56,43,17,42,42,93,60,56,25,33,30,46,How tall do cucumbers grow?


### 2.4 Normalizing text
In some cases (and where the algorithm doesn't already do it) results can be improved by removing stop words and comparing the lemma of each word.


In [ ]:
!pip install nltk | tail -n 1

In [ ]:
import nltk
nltk.download( "averaged_perceptron_tagger" )
nltk.download( "stopwords" )
nltk.download( "wordnet" )

In [22]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re

g_stopwords_arr = stopwords.words( "english" )
g_lemmatizer = WordNetLemmatizer()

def cleanQuestion( question_txt ):
    question_txt = question_txt.lower()
    question_txt = re.sub( r"[^a-z]", " ", question_txt )
    question_txt = re.sub( r"\s+", " ", question_txt )
    return question_txt

def getWordnetPOS( tag ):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def lemmatize( words_in ):
    lemmas_arr = []
    pos_dict = nltk.pos_tag( words_in )
    for index, tag in enumerate( pos_dict ):
        word = words_in[index]
        #if word in g_stopwords_arr:
        #    continue
        pos_info = tag[1]
        lemma = g_lemmatizer.lemmatize( word, getWordnetPOS( pos_info ) )
        lemmas_arr.append( lemma )
    return lemmas_arr

def lemmatizeQuestion( question ):
    question_txt = question if( type( question ) == str ) else question["question_txt"]
    question_txt = cleanQuestion( question_txt )
    question_words = question_txt.split()
    final_words = lemmatize( question_words )
    final_question = " ".join( final_words )
    if( type( question ) == str ):
        return final_question
    new_question = question.copy()
    new_question["question_txt"] = final_question
    return new_question

def normalizeQuestion( question ):
    question_txt = question if( type( question ) == str ) else question["question_txt"]
    question_txt = cleanQuestion( question_txt )
    question_words = question_txt.split()
    lematized_words = lemmatize( question_words )
    final_words = []
    for word in lematized_words:
        if word not in g_stopwords_arr:
            final_words.append( word )
    final_question = " ".join( final_words )
    if( type( question ) == str ):
        return final_question
    new_question = question.copy()
    new_question["question_txt"] = final_question
    return new_question

In [23]:
lemmatizeQuestion( "How tall do tomatoes grow, when the conditions are best?" )

'how tall do tomato grow when the condition be best'

In [24]:
normalizeQuestion( "How tall do tomatoes grow, when the conditions are best?" )

'tall tomato grow condition best'

In [25]:
g_curated_answers_lemmatized = [ lemmatizeQuestion( question ) for question in g_curated_answers ]
g_curated_answers_normalized = [ normalizeQuestion( question ) for question in g_curated_answers ]

In [26]:
import pandas as pd
run_time_question = lemmatizeQuestion( g_run_time_questions[0]["question_txt"] )
results = getFuzzScores( run_time_question, g_curated_answers_lemmatized )
print( "Run-time question:\n\"" + run_time_question + "\"\n" )
print( "Match scores:" )
pd.DataFrame( results )

Run-time question:
"can tomato be grow in pot"

Match scores:


,curated_id,s1,s2,s3,s4,s5,ave_score,curated_question_txt
0,curated_answer_02,58,64,69,84,73,69,can i grow tomato in container
1,curated_answer_01,48,56,52,52,58,53,how tall do cucumber grow


In [27]:
import pandas as pd
run_time_question = normalizeQuestion( g_run_time_questions[0]["question_txt"] )
results = getFuzzScores( run_time_question, g_curated_answers_normalized )
print( "Run-time question:\n\"" + run_time_question + "\"\n" )
print( "Match scores:" )
pd.DataFrame( results )

Run-time question:
"tomato grow pot"

Match scores:


,curated_id,s1,s2,s3,s4,s5,ave_score,curated_question_txt
0,curated_answer_02,50,60,61,85,85,68,grow tomato container
1,curated_answer_01,42,50,42,48,58,48,tall cucumber grow


## 3. Test matching sample questions
The best way to know which methods are most effective for the types of questions your solution gets is to test them.

The function below applies the string comparison methods above to the sample questions.

In [28]:
dispatcher = { 
    "fuzz_" : getFuzzScores, 
    "sen_"  : getSentenceTransformerScores,
    "td_"   : gettextdistanceScores
}

def applyMethod( method_prefix ):
    tmp_results = {}
    for i in range( len( g_run_time_questions ) ):
        q_id = "q" + str( i )
        run_time_question = g_run_time_questions[i]
        #print( "run_time_question:\n" + json.dumps( run_time_question, indent=3 ) )
        expected_match = run_time_question["expected_match"]
        #result_scores_arr = getFuzzScores( run_time_question["question_txt"], g_curated_answers )
        result_scores_arr = dispatcher[ method_prefix ]( run_time_question["question_txt"], g_curated_answers )
        #print( "result_scores_arr:\n" + json.dumps( result_scores_arr, indent=3 ) )
        for score_result_json in result_scores_arr:
            curated_id = score_result_json["curated_id"]
            for key in score_result_json.keys():
                if not re.match( r"^s", key ):
                    continue
                score = score_result_json[ key ]
                method_name = method_prefix + key
                if( method_name not in tmp_results ):
                    tmp_results[ method_name ] = {}
                if( q_id not in tmp_results[ method_name ] ):
                    tmp_results[ method_name ][ q_id ] = { "expected_match" : expected_match }
                if( expected_match is None ):
                    if( "matched_score" not in tmp_results[ method_name ][ q_id ] ):
                        tmp_results[ method_name ][ q_id ][ "matched_score" ] = score
                    elif( tmp_results[ method_name ][ q_id ][ "matched_score" ] < score ):
                        tmp_results[ method_name ][ q_id ][ "unmatched_score" ] = tmp_results[ method_name ][ q_id ][ "matched_score" ]
                        tmp_results[ method_name ][ q_id ][ "matched_score" ] = score
                    else:
                        tmp_results[ method_name ][ q_id ][ "unmatched_score" ] = score
                elif( curated_id == expected_match ):
                    tmp_results[ method_name ][ q_id ][ "matched_score" ] = score
                else:
                    tmp_results[ method_name ][ q_id ][ "unmatched_score" ] = score
    return tmp_results

In [29]:
tmp_results1 = applyMethod( "fuzz_" )
tmp_results2 = applyMethod( "sen_" )
tmp_results3 = applyMethod( "td_" )

In [37]:
tmp_results1

{'fuzz_s1': {'q0': {'expected_match': 'curated_answer_02',
   'matched_score': 60,
   'unmatched_score': 46},
  'q1': {'expected_match': 'curated_answer_01',
   'matched_score': 56,
   'unmatched_score': 33},
  'q2': {'expected_match': None, 'matched_score': 47, 'unmatched_score': 38},
  'q3': {'expected_match': None, 'matched_score': 47, 'unmatched_score': 43}},
 'fuzz_s2': {'q0': {'expected_match': 'curated_answer_02',
   'matched_score': 63,
   'unmatched_score': 51},
  'q1': {'expected_match': 'curated_answer_01',
   'matched_score': 62,
   'unmatched_score': 38},
  'q2': {'expected_match': None, 'matched_score': 62, 'unmatched_score': 57},
  'q3': {'expected_match': None, 'matched_score': 55, 'unmatched_score': 49}},
 'fuzz_s3': {'q0': {'expected_match': 'curated_answer_02',
   'matched_score': 68,
   'unmatched_score': 47},
  'q1': {'expected_match': 'curated_answer_01',
   'matched_score': 53,
   'unmatched_score': 40},
  'q2': {'expected_match': None, 'matched_score': 54, 'unma

The goal is to find methods that meet the following criteria (if possible):
- For each run time question that has an expected_match, make sure the method gives the highest similarity score for the expected match
- For each run time question that doesn't match any curated questions, make sure the similarity scores are low
- Make sure the highest score for unmatched questions is lower than the lowest score for matched questions

The following function checks those criteria and then returns results in a DataFrame:

In [32]:
def resultDataFrame( tmp_results ):
    method_names_arr = tmp_results.keys()
    final_results = []
    for method_name in method_names_arr:
        result_json = tmp_results[ method_name ]
        method_results = { "method" : method_name }
        match_min = 100
        noise_max = 0
        fits = True
        for q in result_json.keys():
            expected_match = result_json[ q ]["expected_match"]
            matched_score = result_json[ q ]["matched_score"]
            unmatched_score = result_json[ q ]["unmatched_score"]
            method_results[ q ] = str( matched_score ) + " / " + str( unmatched_score )
            if( expected_match is not None ):
                if( matched_score < match_min ):
                    match_min = matched_score
                if( unmatched_score > noise_max ):
                    noise_max = unmatched_score
                if( matched_score > unmatched_score ):
                    correct = "✓"
                else:
                    correct = "X"
                    fits = False
                method_results[ q ] += " " + correct
            if( ( expected_match is None ) and ( max( [ matched_score, unmatched_score ] ) > noise_max ) ):
                noise_max = max( [ matched_score, unmatched_score ] )
        method_results["match_min"] = match_min
        method_results["noise_max"] = noise_max
        method_results["method_fits_data"] = "YES" if ( ( fits == True ) and ( match_min > noise_max ) ) else "-"
        final_results.append( method_results )
    return pd.DataFrame( final_results )

In [33]:
resultDataFrame( tmp_results1 )

,method,q0,q1,q2,q3,match_min,noise_max,method_fits_data
0,fuzz_s1,60 / 46 ✓,56 / 33 ✓,47 / 38,47 / 43,56,47,YES
1,fuzz_s2,63 / 51 ✓,62 / 38 ✓,62 / 57,55 / 49,62,62,-
2,fuzz_s3,68 / 47 ✓,53 / 40 ✓,54 / 42,61 / 35,53,61,-
3,fuzz_s4,74 / 47 ✓,53 / 40 ✓,54 / 51,64 / 39,53,64,-
4,fuzz_s5,71 / 51 ✓,62 / 42 ✓,56 / 54,67 / 42,62,67,-


In [34]:
resultDataFrame( tmp_results2 )

,method,q0,q1,q2,q3,match_min,noise_max,method_fits_data
0,sen_score,79 / 37 ✓,78 / 23 ✓,62 / 45,68 / 37,78,68,YES


In [35]:
resultDataFrame( tmp_results3 )

,method,q0,q1,q2,q3,match_min,noise_max,method_fits_data
0,td_s01,12 / 10 ✓,48 / 6 ✓,7 / 0,14 / 8,12,14,-
1,td_s03,42 / 20 ✓,48 / 21 ✓,26 / 26,32 / 24,42,32,YES
2,td_s04,42 / 20 ✓,48 / 21 ✓,26 / 26,32 / 24,42,32,YES
3,td_s05,85 / 61 ✓,69 / 60 ✓,62 / 51,73 / 66,69,73,-
4,td_s06,76 / 61 ✓,69 / 60 ✓,62 / 51,73 / 66,69,73,-
5,td_s07,86 / 64 ✓,82 / 69 ✓,67 / 59,72 / 69,82,72,YES
6,td_s08,61 / 53 ✓,74 / 50 ✓,48 / 38,56 / 53,61,56,YES
7,td_s09,69 / 56 ✓,74 / 56 ✓,54 / 53,59 / 57,69,59,YES
8,td_s10,37 / 19 ✓,48 / 15 ✓,26 / 12,18 / 15,37,26,YES
9,td_s11,75 / 43 ✓,64 / 46 ✓,46 / 38,63 / 45,64,63,YES
